In [1]:
import pandas as pd
import sqlite3


In [2]:
caminho = rf"C:\Users\Arthur\Downloads\reclamacoes-n1e2-distribuidoras-2024.csv"

In [3]:
chunksize = 1_000_000
lista = []
i = 0

for chunk in pd.read_csv(caminho, encoding='latin1', sep=';', chunksize=chunksize):
    i += 1
    print(f"Lendo partição {i}")
    lista.append(chunk)

df_total = pd.concat(lista)
df_total.columns = df_total.columns.str.lower()
df_total.drop(columns = ['numouvprazomediosolucaoimproc', 'numsacprazomediosolucaoimproc'], inplace=True)
novas_colunas = ['data_geracao_dados', 'data_registro', 'distribuidora', 'doc_fiscal_distribuidora', 'codigo_municipio', 'municipio', 'uf', 'regiao', 'classificacao_agente', 'codigo_reclamacao', 'canal_reclamacao', 'reclamacao', 'qtd_reclamacoes_recebidas', 'qtd_reclamacoes_improcedentes', 'qtd_reclamacoes_procedentes', 'prazo_solucao', 'canal_atendimento']
df_total.columns = novas_colunas

df_total['data_registro'] = pd.to_datetime(df_total['data_registro'], errors='coerce')
df_total['ano'] = df_total['data_registro'].dt.year
df_total['mes'] = df_total['data_registro'].dt.month
conn = sqlite3.connect("data/aneel.db")
df_total.to_sql("reclamacoes", conn, if_exists="replace", index=False)
conn.close()
df_estado = df_total.groupby('uf')['qtd_reclamacoes_recebidas'].sum().reset_index()
df_mun = df_total.groupby('municipio')['qtd_reclamacoes_recebidas'].sum().nlargest(10).reset_index()
df_canal = df_total['canal_reclamacao'].value_counts().reset_index()


Lendo partição 1
Lendo partição 2
Lendo partição 3
Lendo partição 4
Lendo partição 5
Lendo partição 6
Lendo partição 7
Lendo partição 8
Lendo partição 9


In [4]:
df_total["qtd_reclamacoes_recebidas"].sum()

np.int64(1179969198)

In [5]:
mascara = df_total['prazo_solucao'].str.startswith(',', na=False)
df_total.loc[mascara, 'prazo_solucao'] = '0' + df_total.loc[mascara, 'prazo_solucao']

In [6]:
df_total["prazo_solucao"] = df_total["prazo_solucao"].str.replace(",",".")
df_total["prazo_solucao"] = df_total["prazo_solucao"].astype(float)
df_total["prazo_solucao"].mean()

np.float64(12.448313894174369)

In [15]:
df_total

,data_geracao_dados,data_registro,distribuidora,doc_fiscal_distribuidora,codigo_municipio,municipio,uf,regiao,classificacao_agente,codigo_reclamacao,canal_reclamacao,reclamacao,qtd_reclamacoes_recebidas,qtd_reclamacoes_improcedentes,qtd_reclamacoes_procedentes,prazo_solucao,canal_atendimento,ano,mes
0,2025-08-09,2024-02-29,Amazonas Energia,2341467000120,1301159,Careiro da Várzea,AM,N,Concessionária,1020101,Nível 1,Solicitação não atendida ou atrasada,1,0,0,"0,00",Telefônico,2024,2
1,2025-08-09,2024-02-29,Amazonas Energia,2341467000120,1303569,Rio Preto da Eva,AM,N,Concessionária,1020101,Nível 1,Solicitação não atendida ou atrasada,1,0,0,"0,00",Telefônico,2024,2
2,2025-08-09,2024-02-29,Amazonas Energia,2341467000120,1302603,Manaus,AM,N,Concessionária,1020101,Nível 1,Solicitação não atendida ou atrasada,3,0,0,"0,00",Telefônico,2024,2
3,2025-08-09,2024-02-29,Amazonas Energia,2341467000120,1300300,Autazes,AM,N,Concessionária,1020101,Nível 1,Solicitação não atendida ou atrasada,1,0,0,"0,00",Outros,2024,2
4,2025-08-09,2024-02-29,Amazonas Energia,2341467000120,1303403,Parintins,AM,N,Concessionária,1020101,Nível 1,Solicitação não atendida ou atrasada,1,0,1,"6,04",Telefônico,2024,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8021947,2025-08-09,2024-12-31,Sulgipe,13255658000196,2803005,Itabaianinha,SE,NE,Concessionária,10106,Nível 2,Tarifas / Fatura / Faturamento / Cobrança,2,0,0,"0,00",Telefônico,2024,12
8021948,2025-08-09,2024-12-31,Sulgipe,13255658000196,2802809,Indiaroba,SE,NE,Concessionária,10106,Nível 2,Tarifas / Fatura / Faturamento / Cobrança,1,0,0,"0,00",Telefônico,2024,12
8021949,2025-08-09,2024-12-31,Sulgipe,13255658000196,2807600,Umbaúba,SE,NE,Concessionária,10106,Nível 2,Tarifas / Fatura / Faturamento / Cobrança,2,0,0,"0,00",Telefônico,2024,12
8021950,2025-08-09,2024-12-31,Sulgipe,13255658000196,2807402,Tobias Barreto,SE,NE,Concessionária,10106,Nível 2,Tarifas / Fatura / Faturamento / Cobrança,1,0,0,"0,00",Telefônico,2024,12


In [11]:
df_estado

,uf,qtd_reclamacoes_recebidas
0,AC,8857028
1,AL,1884641
2,AM,2743236
3,AP,402355
4,BA,75195641
5,CE,38737010
6,DF,2195638
7,ES,9812417
8,GO,6796839
9,MA,2684544
